In [43]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from wmfdata import hive, charting

In [44]:
charting.set_mpl_style()

In [40]:
active_ed_edit_counts = hive.run("""
with apr_active_eds as (
    select
        event_user_text,
        count(*) as content_edits
    from wmf.mediawiki_history mwh
    inner join wmf_raw.mediawiki_project_namespace_map ns_map
    on 
        mwh.wiki_db = ns_map.dbname and
        mwh.page_namespace = ns_map.namespace
    where
        ns_map.snapshot = "2019-04" and
        mwh.snapshot = "2019-04" and
        mwh.event_entity = "revision" and
        mwh.event_type = "create" and
        mwh.event_timestamp between "2019-04" and "2019-05" and
        mwh.event_user_id != 0 and
        ns_map.namespace_is_content = 1
    group by event_user_text
    having content_edits >= 5
)
select
    event_user_text,
    count(*) as edit_count
from apr_active_eds aae 
inner join wmf.mediawiki_history mwh 
on aae.event_user_text = mwh.event_user_text
where
    mwh.snapshot = "2019-04" and
    mwh.event_entity = "revision" and
    mwh.event_type = "create" and
    not mwh.revision_is_deleted_by_page_deletion
group by mwh.event_user_text
""")

In [41]:
active_ed_edit_counts.query("event_user_text == 'WhatamIdoing'")

,event_user_text,edit_count
76195,WhatamIdoing,99789


In [53]:
active_ed_edit_counts["edit_count"].describe(percentiles=[0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99]).apply(lambda x: format(x, 'f'))

count       84865.000000
mean        22754.283639
std        460146.569539
min             1.000000
1%              3.000000
10%             8.000000
25%            24.000000
50%           249.000000
75%          3699.000000
90%         23987.600000
99%        228777.720000
max      53262319.000000
Name: edit_count, dtype: object

In [52]:
active_ed_edit_counts.query("edit_count < 5")

,event_user_text,edit_count
5,"""luisfernandodelasheras""",2
90,0FEBO,1
149,123portalboy123,3
168,1387barsam,2
211,19113s,3
254,1YoshiSu3,2
292,2019paulkim,2
420,34rar,1
437,3drdprinter,3
536,5meggz,4
